# ChatBot

In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input , Embedding , LSTM , Dense , GlobalMaxPooling1D , Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [10]:
%%writefile content.json
{"interns":
 [
     {
         "tag":"greatings",
         "input":["hello","hi there","nice to meet you","hi","hey there","hey"],
         "responses":[
             "Hi. welcome",
             "Hey yo, do you need help i ?",
             "Welcome aboard , how may i help you ?",
             "ahoy!!",
             "Hi random person , what do you want ?",
             "Hello my person , how can i help you ?",
             "Ha , Glad you showed up . How are you ?"
         ]
     } , 
     
     {
         "tag":"goodbye",
         "input":["Thank You" , "Thanks" , "Thanks for the info" , "bye" , "adios" , "ok.bye" , "goodbye" , "see you later" , "i will catch you later" ],
         "responses":[
             "okay, bye",
             "Have a nice day",
             "Adios",
             "Goodbye",
             "See you later",
             "nice conversation",
             "bye",
             "okay , have a nice day",
             "Take care",
             "We will meet soon",
             "Yeah sure . will talk to you later",
             "Okay , Take Care"
         ]
     } ,
     {
         "tag":"howami",
         "input":["how are you ?" , "i am fine" , "are you fine ?" , "how are things going on ?" , "everything going on okay ?" , "how is every thing is going"],
         "responses":[
             "Yeah , am fine",
             "Every thing is going on well",
             "Doing good . Thanks you are my best friend",
             "Things are great",
             "Yeah Fine , nothing much going in my life",
             "Everythings great",
             "I am doing good . Life is boring . Need to go on an adventure"
         ]
     } ,
     {
         "tag":"whoareyou",
         "input":["who are you ?" , "what are you ?" , "what is your name ?" , "are you a chatbot ?" , "what can i call you ?" , "your name ?" , "are you a bot ? "],
         "responses":[
             "I am a chatbot . You can call me Bchat",
             "I am a Bchat you could ask questions",
             "Bchat as your service",
             "My name is Bchat and yes i am a chat bot",
             "I am here to help you"
         ]
     } ,
     {
         "tag":"whereareyou",
         "input":["Where are you from ?" , "Which country are you from ?" , "Where do you live ?" , "Where are you ?" , "Where do you live in ?" , "which place do you live in"],
         "responses":[
             "I live in southern province",
             "I live in southern province . Pretty sure you would not have heared about that",
             "Souther province . thats where i live in"
         ]
     } ,
     
     {
         "tag":"join",
         "input":["How to get recruit ?" , "How to join ?" , "How do i get recruit ?" , "How do i join ?"],
         "responses":[
             "you need contact the nearest HR office once you have the complete the pre requisites",
             "contact the closet HR office from your location",
             "well. there are some pre-requisites and after you complete them , find the HR office"
             ]
     } ,
     
     {
         "tag":"whereisthehroffice",
         "input":["Where is the HR office located ?" , "location of the HR office ?" , "HR office's location ?" , "Where can i find the HR office ?"],
         "responses":[
             "That's up to you . use your skills",
             "They are present all over the city you live in . find them on your own . I can not give any more information",
             "There are a lot of HR offices in the city you are in . find them"
         ]
     } 
     
 ]
}

Overwriting content.json


## importing data

In [11]:
with open('content.json') as content:
    data1 = json.load(content)